In [2]:
import pandas as pd
import numpy as np
from pyhive import hive

In [3]:
sql='''
    select user_id, interest
    from ks_strategy_dev.user_leaf_class_interest_set_0806_sample2
    lateral view explode(interest_set) a as interest
    '''

In [4]:
cursor = hive.connect().cursor()
cursor.execute("set beacon.conf.var.reset.enable=true")
cursor.execute("set beacon.semantic.analyzer.limitsuffix.number=300000000")
cursor.execute("set hive.smart.router.enable=false")
cursor.execute(sql)
results = cursor.fetchall()
cursor.close()

In [5]:
data = pd.DataFrame(results)
interest_list = data.groupby([0])[1].apply(list)

In [6]:
inte_list = pd.DataFrame(interest_list)
inte_list.head()

,1
0,
1000013878,"[186, 22, 244, 26, 267, 271, 275, 277, 283, 28..."
1000014335,"[243, 267, 285]"
1000027736,"[105, 114, 186, 188, 20, 22, 26, 37, 40, 499, ..."
1000041831,"[244, 267, 271, 310, 320, 321, 338, 508, 530, ..."
1000049488,"[300, 303, 40]"


In [7]:
inte_list = inte_list[1].tolist()
print(len(inte_list))

253145


In [8]:
def max_length(l):
    max_int = 0
    for each in l:
        a = len(each)
        if a > max_int:
            max_int = a
    return max_int
max_int = max_length(inte_list)
print(max_int)

87


In [9]:
def create_C1(data_set):
    C1 = set()
    for t in data_set:
        for item in t:
            item_set = frozenset([item])
            C1.add(item_set)
    return C1

In [10]:
C = create_C1(interest_list)
print(C)
print(len(C))

{frozenset({'443'}), frozenset({'275'}), frozenset({'365'}), frozenset({'476'}), frozenset({'669'}), frozenset({'350'}), frozenset({'156'}), frozenset({'604'}), frozenset({'90'}), frozenset({'542'}), frozenset({'572'}), frozenset({'298'}), frozenset({'716'}), frozenset({'701'}), frozenset({'82'}), frozenset({'707'}), frozenset({'77'}), frozenset({'170'}), frozenset({'432'}), frozenset({'679'}), frozenset({'457'}), frozenset({'615'}), frozenset({'42'}), frozenset({'472'}), frozenset({'32'}), frozenset({'179'}), frozenset({'528'}), frozenset({'493'}), frozenset({'209'}), frozenset({'554'}), frozenset({'69'}), frozenset({'8'}), frozenset({'689'}), frozenset({'721'}), frozenset({'140'}), frozenset({'224'}), frozenset({'447'}), frozenset({'107'}), frozenset({'460'}), frozenset({'48'}), frozenset({'690'}), frozenset({'322'}), frozenset({'137'}), frozenset({'698'}), frozenset({'637'}), frozenset({'491'}), frozenset({'51'}), frozenset({'557'}), frozenset({'712'}), frozenset({'89'}), frozenset(

In [11]:
def is_apriori(Ck_item, Lksub1):
# 任何非频繁的(k-1)项集都不是频繁k项集的子集
    for item in Ck_item:
        sub_Ck = Ck_item - frozenset([item])
        if sub_Ck not in Lksub1:
            return False
    return True

In [12]:
def create_Ck(Lksub1, k):
# 连接步骤
    Ck = set()
    len_Lksub1 = len(Lksub1)
    list_Lksub1 = list(Lksub1)
    for i in range(len_Lksub1):
        for j in range(1, len_Lksub1):
            l1 = list(list_Lksub1[i])
            l2 = list(list_Lksub1[j])
            l1.sort()
            l2.sort()
            if l1[0:k-2] == l2[0:k-2]:
                Ck_item = list_Lksub1[i] | list_Lksub1[j]
                # pruning
                if is_apriori(Ck_item, Lksub1):
                    Ck.add(Ck_item)
    return Ck

In [13]:
C2 = create_Ck(C,2)
print(len(C2))

149331


In [14]:
def generate_Lk_by_Ck(data_set, Ck, min_support, support_data):
# 基于压缩后的Ck，扫描所有事务，对Ck中的每个项进行计数，然后删除不满足最小支持度的项，从而获得频繁k项集
    Lk = set()
    item_count = {}
    for t in data_set:
        for item in Ck:
            if item.issubset(t):
                if item not in item_count:
                    item_count[item] = 1
                else:
                    item_count[item] += 1
    t_num = float(len(data_set))
    for item in item_count:
        if (item_count[item] / t_num) >= min_support:
            Lk.add(item)
            support_data[item] = item_count[item] / t_num
    return Lk, support_data

In [51]:
L1, support_data1= generate_Lk_by_Ck(inte_list, C, 0.01, {})

In [15]:
L2, support_data2 = generate_Lk_by_Ck(inte_list, C2, 0.01, {})

In [17]:
print(L2)
print(len(L2))

{frozenset({'319', '275'}), frozenset({'284', '272'}), frozenset({'277', '275'}), frozenset({'456', '267'}), frozenset({'310', '449'}), frozenset({'595', '312'}), frozenset({'186', '238'}), frozenset({'409', '267'}), frozenset({'22', '272'}), frozenset({'3', '277'}), frozenset({'13', '267'}), frozenset({'284', '381'}), frozenset({'338', '238'}), frozenset({'486', '316'}), frozenset({'338', '337'}), frozenset({'321', '267'}), frozenset({'285', '26'}), frozenset({'146', '267'}), frozenset({'267', '248'}), frozenset({'20', '272'}), frozenset({'270', '312'}), frozenset({'270', '310'}), frozenset({'338', '277'}), frozenset({'199', '267'}), frozenset({'246', '248'}), frozenset({'729', '690'}), frozenset({'542', '275'}), frozenset({'3', '283'}), frozenset({'40', '107'}), frozenset({'271', '13'}), frozenset({'529', '312'}), frozenset({'20', '312'}), frozenset({'271', '321'}), frozenset({'425', '729'}), frozenset({'3', '310'}), frozenset({'271', '669'}), frozenset({'338', '26'}), frozenset({'33

{frozenset({'284', '277'}): 0.0287108179106836, frozenset({'312', '186'}): 0.03743309170633431, frozenset({'271', '283'}): 0.09169448339884256, frozenset({'729', '267'}): 0.17342629718145727, frozenset({'731', '275'}): 0.04665310395228031, frozenset({'26', '284'}): 0.07767090007703095, frozenset({'26', '283'}): 0.09605562029666792, frozenset({'731', '186'}): 0.02851725295779099, frozenset({'3', '284'}): 0.01644117008038871, frozenset({'244', '267'}): 0.021710877165261017, frozenset({'731', '271'}): 0.0834738983586482, frozenset({'312', '26'}): 0.09528531079025855, frozenset({'271', '267'}): 0.37175926840348417, frozenset({'277', '267'}): 0.07923127061565506, frozenset({'312', '277'}): 0.029698394200952022, frozenset({'186', '283'}): 0.06974263761875604, frozenset({'271', '26'}): 0.24636078137036085, frozenset({'312', '267'}): 0.11856840940962689, frozenset({'312', '275'}): 0.04726145094708566, frozenset({'186', '277'}): 0.029342866736455392, frozenset({'283', '277'}): 0.033387979221394

In [20]:
C3 = create_Ck(L2, 3)

In [22]:
print(len(C3))

5738


In [23]:
L3, support_data3 = generate_Lk_by_Ck(inte_list, C3, 0.01, {})

In [24]:
print(len(L3))
print(L3)

2543
{frozenset({'26', '40', '272'}), frozenset({'729', '735', '26'}), frozenset({'285', '283', '381'}), frozenset({'186', '277', '267'}), frozenset({'283', '40', '381'}), frozenset({'20', '312', '186'}), frozenset({'690', '310', '267'}), frozenset({'26', '542', '203'}), frozenset({'486', '40', '277'}), frozenset({'22', '338', '283'}), frozenset({'690', '26', '275'}), frozenset({'729', '69', '267'}), frozenset({'271', '690', '310'}), frozenset({'486', '26', '272'}), frozenset({'20', '735', '267'}), frozenset({'317', '272', '267'}), frozenset({'729', '26', '272'}), frozenset({'320', '731', '271'}), frozenset({'267', '530', '238'}), frozenset({'20', '729', '731'}), frozenset({'26', '283', '530'}), frozenset({'283', '310', '337'}), frozenset({'542', '271', '690'}), frozenset({'243', '26', '267'}), frozenset({'271', '312', '316'}), frozenset({'186', '283', '272'}), frozenset({'312', '40', '243'}), frozenset({'320', '338', '271'}), frozenset({'275', '26', '64'}), frozenset({'338', '690', '2

In [26]:
C4 = create_Ck(L3, 4)

In [28]:
print(len(C4))

6589


In [29]:
L4, support_data4 = generate_Lk_by_Ck(inte_list, C4, 0.01, {})

In [31]:
print(len(L4))

4090


In [32]:
C5 = create_Ck(L4, 5)

In [33]:
print(len(C5))

4317


In [34]:
L5, support_data5 = generate_Lk_by_Ck(inte_list, C5, 0.01, {})

In [37]:
print(len(L5))

3374


In [38]:
C6 = create_Ck(L5, 6)

In [40]:
print(len(C6))

1468


In [42]:
L6, support_data6 = generate_Lk_by_Ck(inte_list, C6, 0.01, {})
print(len(L6))

1233


In [82]:
C7 = create_Ck(L6, 7)

In [83]:
L7, support_data7 = generate_Lk_by_Ck(inte_list, C7, 0.01, {})
print(len(L7))

148


In [84]:
C8 = create_Ck(L7, 8)

In [85]:
L8, support_data8 = generate_Lk_by_Ck(inte_list, C8, 0.01, {})
print(len(L8))

1


In [86]:
print(L8)

{frozenset({'731', '271', '26', '272', '338', '267', '729', '283'})}


In [36]:
def generate_big_rules(L, support_data, min_conf):
    big_rule_list = []
    sub_set_list = []
    for i in range(0, len(L)):
        for freq_set in L[i]:
            for sub_set in sub_set_list:
                if sub_set.issubset(freq_set):
                    conf = support_data[freq_set] / support_data[freq_set - sub_set]
                    big_rule = (freq_set - sub_set, sub_set, conf)
                    if conf >= min_conf and big_rule not in big_rule_list:
#                         print freq_set-sub_set, " => ", sub_set, "conf: ", conf
                        big_rule_list.append(big_rule)
            sub_set_list.append(freq_set)
    return big_rule_list

In [87]:
l = []
l.append(L1)
l.append(L2)
l.append(L3)
l.append(L4)
l.append(L5)
l.append(L6)
l.append(L7)
l.append(L8)

In [88]:
s = {}
s.update(support_data1)
s.update(support_data2)
s.update(support_data3)
s.update(support_data4)
s.update(support_data5)
s.update(support_data6)
s.update(support_data7)
s.update(support_data8)

In [89]:
print(len(s))

12389


In [54]:
big_rule_list = generate_big_rules(l, s, 0.7)

In [56]:
print(len(big_rule_list))

7725


In [90]:
big_rule_list2 = generate_big_rules(l, s, 0.8)

In [91]:
print(len(big_rule_list2))

18678


In [75]:
print(big_rule_list2[0:100])

[(frozenset({'690'}), frozenset({'729'}), 0.82286866359447), (frozenset({'735'}), frozenset({'26'}), 0.8035378246142265), (frozenset({'530'}), frozenset({'267'}), 0.8191534843950407), (frozenset({'16'}), frozenset({'267'}), 0.8040019762845849), (frozenset({'382'}), frozenset({'267'}), 0.8278344331133773), (frozenset({'317'}), frozenset({'267'}), 0.8858817445268057), (frozenset({'257'}), frozenset({'267'}), 0.8870447210696173), (frozenset({'312'}), frozenset({'267'}), 0.8438053470523741), (frozenset({'285'}), frozenset({'267'}), 0.8326891220320265), (frozenset({'719'}), frozenset({'267'}), 0.8029482551143202), (frozenset({'492'}), frozenset({'26'}), 0.8162162162162162), (frozenset({'272'}), frozenset({'267'}), 0.8278508518458587), (frozenset({'529'}), frozenset({'267'}), 0.8539219401288368), (frozenset({'10'}), frozenset({'267'}), 0.8018638573743924), (frozenset({'595'}), frozenset({'267'}), 0.8815068493150686), (frozenset({'451'}), frozenset({'26'}), 0.8388238916256158), (frozenset({'1

In [70]:
print(list(big_rule_list2[0][0]))

['690']


In [92]:
result = []
for each in big_rule_list2:
    rule = []
    rule.append(list(each[0]))
    rule.append(list(each[1]))
    rule.append(each[2])
    result.append(rule)

In [93]:
result = pd.DataFrame(result)
result.head()

,0,1,2
0,[690],[729],0.822869
1,[735],[26],0.803538
2,[530],[267],0.819153
3,[16],[267],0.804002
4,[382],[267],0.827834


In [94]:
output = result.to_csv('interest_rules.csv',index=False)

In [73]:
output = pd.DataFrame(big_rule_list2)

In [74]:
output.head()

,0,1,2
0,(690),(729),0.822869
1,(735),(26),0.803538
2,(530),(267),0.819153
3,(16),(267),0.804002
4,(382),(267),0.827834
